In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pandas.api.types import is_numeric_dtype

In [2]:
def cardinalidad(df_in, umbral_categoria = 10, umbral_countinua = 30):
    cardinalidad = [df_in[col].nunique() for col in df_in.columns]
    cardinalidad_por = [df_in[col].nunique()/len(df_in[col]) for col in df_in.columns]
    dict_df = {"nombre_variable": df_in.columns, "valores_unicos": cardinalidad, "cardinalidad": cardinalidad_por}
    nuevo_df = pd.DataFrame(dict_df)
    nuevo_df["tipo_sugerido"] = "Categórica"
    nuevo_df.loc[nuevo_df["valores_unicos"] == 2, "tipo_sugerido"] = "Binaria"
    nuevo_df.loc[nuevo_df["valores_unicos"] >= umbral_categoria, "tipo_sugerido"] = "Numerica Discreta"
    nuevo_df.loc[nuevo_df["cardinalidad"] >= umbral_countinua, "tipo_sugerido"] = "Numerica Continua"
    return nuevo_df

In [9]:
def get_features_cat_regression(df, target_col, p_value = 0.05, umbral_cat = 10):
    '''
    Esta función filtra las variables categóricas de un dataset introducido para entrenar un modelo de regresión lineal. 
    Verifica el tipo de variable llamando a la función "cardinalidad".
    Si la variable es binaria aplica el test Mann-Whitney U, si es categórica pero no binaria aplica el test ANOVA para comprobar su relación con la variable target.
    Si el valor p de los tests está por debajo del umbral especificado en "p_value" puede descartarse la hipótesis nula (la variable target y la categórica no estan relacionadas) con confianza estadística y se añade la variable a la lista de features categóricas para el modelo.


    
    Argumentos:

    df(pd.DataFrame): DataFrame cuyas variables categóricas se desea filtrar.

    target_col(string): nombre de la columna target que se pretende predecir con el modelo.

    p_value(float): umbral de valor p por debajo del cual debe estar el valor p del test aplicado para determinar la relación entre una variable y el target para añadir dicha variable a la lista de features categóricas.
    
    umbral_cat(int): controla el número máximo de valores que puede tener una columna para ser considerada categórica.

    

    Retorna:

    list: Lista de features categóricas para entrenar un modelo de regresión lineal con el dataset dado.
    '''
    if not is_numeric_dtype(df[target_col]):
        target_col = input("Tu variable objetivo no es de tipo numérico, introduce una nueva variable target o la palabra 'parar' para dejar de ejecutar la función.")
        if target_col == "parar":
            return "La función no se ha ejecutado porque has decidido pararla"
    if len(df.loc[df[target_col].isna()]) > 0:
        raise Exception(f"La variable '{target_col}' tiene valores nulos, introduce una variable target sin nulos")    
    from scipy.stats import mannwhitneyu
    from scipy.stats import f_oneway
    lista_cat = []
    for col in df:
        if len(df.loc[df[col].isna()]) > 0:
            raise Exception(f"La variable '{col}' tiene valores nulos, introduce un DataFrame sin nulos")
        tipo_col = cardinalidad(df[[col]], umbral_categoria = umbral_cat).tipo_sugerido[0]
        if tipo_col == "Binaria":
            value_1 = df[col].unique()[0]
            value_2 = df[col].unique()[1]
            group_a = df.loc[df[col] == value_1, target_col]
            group_b = df.loc[df[col] == value_2, target_col]
            _, p_val = mannwhitneyu(group_a, group_b)
            print(f"Para '{target_col}' y '{col}' el p-value es: {p_val} (Test realizado: Mann-Whitney U)")
            print(p_val)
            if p_val < p_value:
                lista_cat.append(col)
        elif tipo_col == "Categórica":
            groups = df[col].unique()
            target_values_x_group = [df.loc[df[col] == group, target_col] for group in groups]
            _, p_val = f_oneway(*target_values_x_group)
            print(f"Para '{target_col}' y '{col}' el p-value es: {p_val} (Test aplicado: ANOVA)")
            if p_val < p_value:
                lista_cat.append(col)
    return lista_cat
    

In [4]:
df_titanic = pd.read_csv("./data/titanic.csv")
df_titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [5]:
cardinalidad(df_titanic[["who"]]).tipo_sugerido[0]

'Categórica'

In [6]:
is_numeric_dtype(df_titanic["sex"])

False

In [10]:
get_features_cat_regression(df_titanic.dropna(), "age")

Para 'age' y 'survived' el p-value es: 0.0006641019760370105 (Test realizado: Mann-Whitney U)
0.0006641019760370105
Para 'age' y 'pclass' el p-value es: 0.00010470399891167751 (Test aplicado: ANOVA)
Para 'age' y 'sex' el p-value es: 0.008040695758472983 (Test realizado: Mann-Whitney U)
0.008040695758472983
Para 'age' y 'sibsp' el p-value es: 0.0894492551676653 (Test aplicado: ANOVA)
Para 'age' y 'parch' el p-value es: 9.908210573457945e-06 (Test aplicado: ANOVA)
Para 'age' y 'embarked' el p-value es: 0.470768684811386 (Test aplicado: ANOVA)
Para 'age' y 'class' el p-value es: 0.00010470399891167751 (Test aplicado: ANOVA)
Para 'age' y 'who' el p-value es: 2.4733196205109136e-16 (Test aplicado: ANOVA)
Para 'age' y 'adult_male' el p-value es: 1.4385968017342073e-05 (Test realizado: Mann-Whitney U)
1.4385968017342073e-05
Para 'age' y 'deck' el p-value es: 0.00021171214067990663 (Test aplicado: ANOVA)
Para 'age' y 'embark_town' el p-value es: 0.470768684811386 (Test aplicado: ANOVA)
Para 'a

['survived',
 'pclass',
 'sex',
 'parch',
 'class',
 'who',
 'adult_male',
 'deck',
 'alive',
 'alone']

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway

In [ ]:
len(df.loc[df[col].isna() > 0])

NameError: name 'df' is not defined

In [ ]:
df_titanic.loc[df_titanic.embark_town.isna()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
61,1,1,female,38.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True
829,1,1,female,62.0,0,0,80.0,NaN,First,woman,False,B,NaN,yes,True


In [ ]:
groups = df_titanic.embark_town.unique()
target_values_x_group = [df_titanic.dropna().loc[df_titanic.embark_town == group, "age"] for group in groups]
_, p_val = f_oneway(*target_values_x_group)
p_val

C:\Users\dario\AppData\Local\Temp\ipykernel_5968\851147082.py:3: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p_val = f_oneway(*target_values_x_group)


np.float64(nan)

In [ ]:
df_titanic.dropna(inplace=True)

In [ ]:
value_1 = df_titanic.dropna()["embark_town"].unique()[0]
value_2 = df_titanic.dropna()["embark_town"].unique()[1]
group_a = df_titanic.dropna().loc[df_titanic["embark_town"] == value_1, "age"]
group_b = df_titanic.dropna().loc[df_titanic["embark_town"] == value_2, "age"]
p, p_val = mannwhitneyu(group_a, group_b)
print(p_val)

0.3389044283200656


In [ ]:
value_1 = df_titanic["survived"].unique()[0]
value_1
group_a = df_titanic.loc[df_titanic["survived"] == value_1]["age"]
group_a
group_b = df_titanic.loc[df_titanic["survived"] == value_2, "age"]
group_a

0      22.0
4      35.0
5       NaN
6      54.0
7       2.0
       ... 
884    25.0
885    39.0
886    27.0
888     NaN
890    32.0
Name: age, Length: 549, dtype: float64